In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer
from gensim import matutils, models
import scipy.sparse

In [7]:
df = pd.read_csv('/content/drive/MyDrive/Amazon_reviews_after_sentiment.csv')
df.head()

,asin,rating,reviews,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word,Good_reviews,polarity,subjectivity
0,B07HBDNGHC,4.0,auf die funktion der box gehe ich nur kurz ein...,165,43,0,3,0,352,2108,5.0,1.0,0.23,0.59
1,B07HBDNGHC,4.0,ich hielt die toniebox immer für einen überteu...,542,174,0,6,7,1135,7647,5.7,1.0,0.50,0.86
2,B07HBDNGHC,5.0,die box ist ein hübsches gerät leider ist sie ...,29,9,0,1,0,57,333,4.9,1.0,0.00,0.00
3,B07HBDNGHC,5.0,hier muss ich einfach bewertenbin super zufrie...,27,9,0,1,0,56,334,5.0,1.0,0.33,0.67
4,B07HBDNGHC,5.0,wir lieben die toniebox sie ist mega praktisch...,9,3,0,0,0,24,159,5.7,1.0,0.33,0.67


In [10]:
df.isnull().sum()

asin            0
rating          0
reviews         0
stopwords       0
punctuation     0
hastags         0
numerics        0
upper           0
word_count      0
char_count      0
avg_word        0
Good_reviews    0
polarity        0
subjectivity    0
dtype: int64

In [9]:
df= df.dropna()

Document-Term-Matrix

In [11]:
df.shape

(5878, 14)

In [12]:
df_good_reviews= df.loc[df.Good_reviews ==1][0:10000]

In [14]:
cv = CountVectorizer()
data_cv = cv.fit_transform(df_good_reviews.reviews)

In [15]:
data_cv.shape

(3161, 21244)

In [16]:
data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = df_good_reviews.index

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
data = pd.read_pickle('dtm_stop.pkl')
data

,00110,00112,01,02,02022017,03,04,05,08082019,10,...,übt,übung,übungen,übungsmodus,übungübrigens,üppigem,üppigen,üüüberall,świetna,şekilde
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5871,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5872,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5873,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5874,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
tdm = data.transpose()
tdm.head()
     

,0,1,2,3,4,5,6,7,8,9,...,5866,5867,5868,5869,5870,5871,5872,5873,5874,5875
00110,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02022017,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [22]:
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [23]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.030*"und" + 0.026*"die" + 0.020*"ist" + 0.016*"das" + 0.016*"der" + 0.016*"es" + 0.014*"für" + 0.011*"zu" + 0.011*"auch" + 0.010*"man"'),
 (1,
  '0.007*"la" + 0.005*"de" + 0.004*"un" + 0.004*"set" + 0.004*"di" + 0.004*"con" + 0.004*"una" + 0.003*"que" + 0.003*"en" + 0.003*"tablet"')]

In [24]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.030*"die" + 0.025*"und" + 0.020*"der" + 0.017*"ist" + 0.015*"das" + 0.012*"man" + 0.011*"es" + 0.010*"nicht" + 0.010*"mit" + 0.009*"zu"'),
 (1,
  '0.032*"und" + 0.022*"die" + 0.021*"ist" + 0.019*"es" + 0.017*"das" + 0.016*"für" + 0.012*"der" + 0.012*"zu" + 0.012*"auch" + 0.012*"sehr"'),
 (2,
  '0.009*"la" + 0.007*"super" + 0.007*"de" + 0.006*"sehr" + 0.006*"set" + 0.005*"un" + 0.005*"gut" + 0.005*"di" + 0.005*"con" + 0.005*"una"')]

In [25]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.028*"die" + 0.028*"und" + 0.021*"ist" + 0.017*"der" + 0.015*"ich" + 0.014*"es" + 0.013*"für" + 0.013*"das" + 0.012*"auch" + 0.011*"nicht"'),
 (1,
  '0.021*"und" + 0.016*"das" + 0.013*"der" + 0.012*"die" + 0.011*"für" + 0.011*"ist" + 0.011*"ein" + 0.010*"mit" + 0.008*"man" + 0.006*"eine"'),
 (2,
  '0.011*"la" + 0.010*"set" + 0.008*"de" + 0.007*"un" + 0.006*"con" + 0.006*"di" + 0.006*"una" + 0.005*"lego" + 0.005*"gut" + 0.005*"que"'),
 (3,
  '0.032*"und" + 0.026*"die" + 0.019*"ist" + 0.019*"es" + 0.018*"das" + 0.016*"zu" + 0.015*"spiel" + 0.015*"der" + 0.013*"für" + 0.011*"man"')]

In [ ]:
#Only nouns

In [26]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
import string
from nltk import word_tokenize, pos_tag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [27]:
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)
     

In [29]:
data_nouns = pd.DataFrame(df_good_reviews.reviews.apply(nouns))
data_nouns

,reviews
0,auf die funktion der box gehe nur kurz ein da ...
1,toniebox immer für einen überteuerten mit über...
2,die box ist ein leider ist sie wie man einem g...
3,muss einfach bewertenbin zufrieden hab die ton...
4,wir lieben die toniebox ist praktisch super le...
...,...
5871,artikel bekannt guter ravensburger qualität gu...
5872,da können die günstigen anbieter mithaltensupe...
5873,da geschenk keine testung möglichgeschenk
5874,schnelle lieferung hervorragende qualität perd...


In [32]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer


# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ["da","zu","auf","man", "e", "den", "spiel","und","auch","für", "mit", 
         "ein","eine","wie", "die", "der", "ist", "sind", "oder", "aber", "das", "alle", "nicht", "war", 
                   "kann",'die','und','ist','das','der','es','nicht','für','zu','ich','man','auch','ein','mit','den','sehr',
                   'aber','spiel','sich','sind','hat','auf','eine','wir','von','kann','nur','gut','bei','war','haben','dem',
                   'als','dass', 'da','wie', 'sie', 'noch','leider', 'im', 'zum', 'schon','wenn','nach','kinder','aus','oder',
                   'habe','dann','viel']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)


# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.reviews)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,12jähriges,a3,a6,aaa,aabatterien,ab,abastecer,abbastanza,abbaue,abbauender,...,übrigens,übrigsuper,übt,übung,übungen,übungsmodus,üppigem,üppigen,świetna,şekilde
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5871,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5872,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5873,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5874,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [34]:
# 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.007*"spaß" + 0.005*"wird" + 0.005*"einen" + 0.005*"immer" + 0.005*"macht" + 0.005*"mir" + 0.005*"einfach" + 0.005*"etwas" + 0.005*"preis" + 0.005*"mehr"'),
 (1,
  '0.006*"tolles" + 0.005*"alles" + 0.005*"lego" + 0.004*"geschenk" + 0.004*"qualität" + 0.004*"mein" + 0.004*"schönes" + 0.004*"kind" + 0.004*"begeistert" + 0.004*"schöne"')]

In [35]:
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.005*"immer" + 0.005*"einfach" + 0.004*"damit" + 0.004*"mehr" + 0.004*"mir" + 0.004*"kleine" + 0.004*"lego" + 0.004*"geschenk" + 0.004*"macht" + 0.004*"super"'),
 (1,
  '0.008*"spaß" + 0.005*"etwas" + 0.005*"macht" + 0.005*"alles" + 0.005*"einen" + 0.005*"einfach" + 0.005*"wird" + 0.005*"mir" + 0.004*"immer" + 0.004*"wieder"'),
 (2,
  '0.006*"mein" + 0.005*"meine" + 0.005*"kind" + 0.005*"lego" + 0.005*"wird" + 0.004*"qualität" + 0.004*"spaß" + 0.004*"etwas" + 0.004*"tochter" + 0.004*"immer"')]

In [38]:
# Only nouns model
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=80)
ldan.print_topics()
     

[(0,
  '0.008*"spaß" + 0.008*"meine" + 0.007*"macht" + 0.007*"alles" + 0.007*"immer" + 0.006*"mein" + 0.006*"geschenk" + 0.006*"tochter" + 0.006*"kind" + 0.005*"tolles"'),
 (1,
  '0.005*"la" + 0.004*"que" + 0.004*"di" + 0.003*"tablet" + 0.003*"che" + 0.003*"amazon" + 0.003*"tonie" + 0.002*"il" + 0.002*"pasta" + 0.002*"geschichte"'),
 (2,
  '0.007*"einen" + 0.005*"einfach" + 0.005*"mir" + 0.005*"etwas" + 0.005*"spaß" + 0.005*"wird" + 0.005*"wieder" + 0.005*"gibt" + 0.005*"immer" + 0.005*"mehr"')]

Topics for bad reviews

In [39]:
df_good_reviews= df.loc[df.Good_reviews ==0][0:10000]

In [40]:
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
data_cv = cv.fit_transform(df_good_reviews.reviews)

data_stop = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_stop.index = df_good_reviews.index

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [41]:
pickle.dump(cv, open("cv_stop.pkl", "wb"))
data_stop.to_pickle("dtm_stop.pkl")

# Let's read in our document-term matrix
data = pd.read_pickle('dtm_stop.pkl')
data

,00,001s,005,007,009,022022,05cm,06l,0815smartphone,09,...,übrigdie,übrigen,übrigens,übt,übung,übungssache,üppigen,české,češtinanemá,że
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5876,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
tdm = data.transpose()
tdm.head()

,10,11,12,13,14,15,16,17,18,19,...,5859,5860,5861,5862,5863,5864,5865,5876,5877,5878
00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001s,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [44]:
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [45]:
#number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()
     

[(0,
  '0.004*"die" + 0.004*"und" + 0.003*"der" + 0.003*"la" + 0.003*"box" + 0.003*"lego" + 0.002*"de" + 0.002*"un" + 0.002*"mit" + 0.002*"lok"'),
 (1,
  '0.028*"die" + 0.025*"und" + 0.020*"das" + 0.019*"ist" + 0.018*"nicht" + 0.017*"der" + 0.016*"es" + 0.012*"ich" + 0.011*"zu" + 0.010*"man"')]

In [46]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.006*"la" + 0.005*"un" + 0.005*"di" + 0.004*"il" + 0.004*"per" + 0.004*"non" + 0.004*"de" + 0.003*"lego" + 0.003*"le" + 0.003*"che"'),
 (1,
  '0.003*"es" + 0.003*"lego" + 0.002*"box" + 0.002*"para" + 0.002*"set" + 0.002*"el" + 0.002*"que" + 0.002*"one" + 0.002*"bond" + 0.002*"lo"'),
 (2,
  '0.030*"die" + 0.026*"und" + 0.020*"das" + 0.020*"ist" + 0.019*"nicht" + 0.018*"der" + 0.016*"es" + 0.013*"ich" + 0.011*"zu" + 0.010*"man"')]

Only nouns

In [48]:
data_nouns = pd.DataFrame(df_good_reviews.reviews.apply(nouns))
data_nouns

,reviews
10,wenn man im bekanntenkreis immer ein wort man ...
11,das seine toniebox weil es keine kinder konnte...
12,ließ mit dem wlan war nicht das erste mit dies...
13,zu aller der händler bei dem wir die box beste...
14,leider toniebox nach einem halben jahr bereits...
...,...
5864,trop petitsje sais pas quoi faire rennes
5865,
5876,die farben waren dass sie für meine tochter an...
5877,amazon aus dem programm ungeeignet


In [49]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer


# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ["da","zu","auf","man", "e", "den", "spiel","und","auch","für", "mit", 
         "ein","eine","wie", "die", "der", "ist", "sind", "oder", "aber", "das", "alle", "nicht", "war", 
                   "kann",'die','und','ist','das','der','es','nicht','für','zu','ich','man','auch','ein','mit','den','sehr',
                   'aber','spiel','sich','sind','hat','auf','eine','wir','von','kann','nur','gut','bei','war','haben','dem',
                   'als','dass', 'da','wie', 'sie', 'noch','leider', 'im', 'zum', 'schon','wenn','nach','kinder','aus','oder',
                   'habe','dann','viel']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)


# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.reviews)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,a3,a4,a4format,aa,aaaakku,aaabatterie,aaaber,aabatterienbatterien,ab,aballerdings,...,übrigdie,übrigen,übrigens,übt,übung,übungssache,üppigen,české,češtinanemá,że
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5876,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5877,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [51]:
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.007*"mehr" + 0.005*"mir" + 0.005*"wird" + 0.005*"etwas" + 0.004*"wieder" + 0.004*"schnell" + 0.004*"muss" + 0.004*"einen" + 0.004*"ganz" + 0.004*"tochter"'),
 (1,
  '0.005*"wieder" + 0.005*"mehr" + 0.004*"qualität" + 0.004*"verpackung" + 0.003*"des" + 0.003*"waren" + 0.003*"immer" + 0.003*"preis" + 0.003*"einen" + 0.003*"mir"')]

In [52]:
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.009*"mehr" + 0.006*"wieder" + 0.006*"wird" + 0.005*"schnell" + 0.005*"etwas" + 0.005*"mir" + 0.005*"einen" + 0.005*"muss" + 0.004*"immer" + 0.004*"hatte"'),
 (1,
  '0.008*"lego" + 0.005*"des" + 0.004*"ganz" + 0.003*"lok" + 0.002*"alles" + 0.002*"box" + 0.002*"product" + 0.002*"teile" + 0.002*"viele" + 0.002*"la"'),
 (2,
  '0.006*"box" + 0.005*"mir" + 0.004*"keine" + 0.004*"preis" + 0.004*"di" + 0.003*"lego" + 0.003*"qualität" + 0.003*"il" + 0.003*"wieder" + 0.003*"zeit"')]

In [54]:
# final model with nouns
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=80)
ldan.print_topics()

[(0,
  '0.005*"mir" + 0.005*"flasche" + 0.003*"lok" + 0.003*"durch" + 0.003*"einer" + 0.003*"des" + 0.003*"sein" + 0.003*"einen" + 0.003*"fernbedienung" + 0.003*"ganz"'),
 (1,
  '0.007*"la" + 0.006*"il" + 0.006*"di" + 0.005*"lego" + 0.005*"non" + 0.005*"che" + 0.004*"box" + 0.004*"que" + 0.004*"product" + 0.003*"le"'),
 (2,
  '0.008*"mehr" + 0.007*"wieder" + 0.006*"tochter" + 0.006*"meine" + 0.005*"box" + 0.004*"keine" + 0.004*"allerdings" + 0.003*"damit" + 0.003*"macht" + 0.003*"spaß"'),
 (3,
  '0.008*"mehr" + 0.006*"etwas" + 0.006*"mir" + 0.006*"wird" + 0.005*"schnell" + 0.005*"muss" + 0.005*"einfach" + 0.005*"wieder" + 0.005*"des" + 0.005*"einen"')]